In [7]:
# ╔══════════════════════════════════════════╗
# ║  UPPGIFT 4 – Full backtest 2015 → 2025     ║
# ╚══════════════════════════════════════════╝

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Välj aktie (byt hur mycket du vill sen!)
ticker = "TSLA"
data = yf.download(ticker, start="2015-01-01", end="2025-11-26", progress=False)

# Glidande medelvärden
short_window = 20
long_window  = 50
data['SMA_short'] = data['Close'].rolling(short_window).mean()
data['SMA_long']  = data['Close'].rolling(long_window).mean()

# Signaler
data['Signal'] = 0
data['Signal'][long_window:] = np.where(
    data['SMA_short'][long_window:] > data['SMA_long'][long_window:], 1, 0)
data['Position'] = data['Signal'].diff()   # 1 = köp, -1 = sälj

# ───── BACKTEST ─────
start_kapital = 100_000
portfolio = pd.DataFrame(index=data.index)
portfolio['cash']     = Start kapital
portfolio['holdings'] = 0          # antal aktier
portfolio['total']    = Start kapital
portfolio['returns']  = 0.0

aktier = 0
for i in range(1, len(data)):
    pris = data['Close'].iloc[i]
    datum = data.index[i]
    
    # Köp
    if data['Position'].iloc[i] == 1:
        aktier = portfolio['cash'].iloc[i-1] // pris
        kostnad = aktier * pris
        portfolio.loc[datum:, 'cash'] = portfolio['cash'].iloc[i-1] - kostnad
        portfolio.loc[datum:, 'holdings'] = aktier
    
    # Sälj
    elif data['Position'].iloc[i] == -1 and aktier > 0:
        intäkt = aktier * pris
        portfolio.loc[datum:, 'cash'] = portfolio['cash'].iloc[i-1] + intäkt
        portfolio.loc[datum:, 'holdings'] = 0
        aktier = 0
    
    # Uppdatera totalt värde
    portfolio.loc[datum, 'total'] = portfolio['cash'].iloc[i] + (aktier * pris)

portfolio = portfolio.ffill()

# ───── RESULTAT ─────
slutvärde = portfolio['total'].iloc[-1]
total_avkastning = (slutvärde / start_kapital - 1) * 100
årlig_avkastning = ((slutvärde/start_kapital)**(1/10.9) - 1) * 100

print(f"\n{ticker} 2015 → 2025")
print(f"Startkapital:     100 000 kr")
print(f"Slutkapital:      {slutvärde:,.0f} kr")
print(f"Total avkastning: {total_avkastning:.1f} %")
print(f"Årlig avkastning: {årlig_avkastning:.1f} %")

# Jämför med köp-och-behåll
bh_avkastning = (data['Close'].iloc[-1] / data['Close'].iloc[long_window] - 1) * 100
print(f"Köp-och-behåll:   {bh_avkastning:.1f} %")

SyntaxError: invalid syntax (1163578929.py, line 29)

In [8]:
# ───── GRAF ─────
plt.figure(figsize=(16,9))
plt.plot(portfolio.index, portfolio['total'],
         label=f"Strategi → {slutvärde:,.0f} kr (+{total_avkastning:.1f}%)", linewidth=3)

# Köp-och-behåll-kurva
bh_value = start_kapital * data['Close'] / data['Close'].iloc[long_window]
plt.plot(data.index, bh_value, label="Köp-och-behåll", alpha=0.7, linewidth=2)

# Signaler
köp = data[data['Position'] == 1].index
sälj = data[data['Position'] == -1].index
plt.plot(köp, portfolio.loc[köp, 'total'], '^', markersize=14, color='green', label='KÖP')
plt.plot(sälj, portfolio.loc[sälj, 'total'], 'v', markersize=14, color='red', label='SÄLJ')

plt.title(f"Backtest {ticker} → 100k blev {slutvärde:,.0f} kr", fontsize=20)
plt.ylabel("Portföljvärde (SEK)")
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

NameError: name 'slutvärde' is not defined

<Figure size 1600x900 with 0 Axes>